In [2]:
print('gola')

gola


In [3]:
import os

import cv2
import numpy as np
from moviepy.video.io.VideoFileClip import VideoFileClip

# import speech_recognition as sr

def analyze_video(video_path, output_dir="output"):
    """
    Analyze video file: extract frames, audio, and generate transcription
    Parameters:
        video_path (str): Path to the video file
        output_dir (str): Directory to save extracted content
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    def extract_frames():
        """Extract frames from video and save key frames"""
        cap = cv2.VideoCapture(video_path)
        frame_count = 0
        prev_frame = None
        frame_exported_count = 0

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Save frame if it's significantly different from previous frame
            if prev_frame is not None:
                diff = np.mean(np.abs(frame - prev_frame))
                if diff > 135:  # Threshold for scene change
                    frame_path = f"{output_dir}/frame_{frame_count:04d}.jpg"
                    cv2.imwrite(frame_path, frame)
                    frame_exported_count = frame_exported_count + 1
            prev_frame = frame.copy()
            frame_count += 1

        cap.release()
        print(f"Total Extracted {frame_exported_count} frames")
        return frame_count

    def extract_audio():
        """Extract audio from video and save as WAV with reduced size"""
        video = VideoFileClip(video_path)
        audio = video.audio

        # Convert to mono and reduce sample rate
        audio_path = f"{output_dir}/audio.mp3"  # Changed extension to .mp3
        audio.write_audiofile(
            audio_path,
            fps=16000,  # Reduce sample rate from 44100 to 16000 Hz
            nbytes=2,   # 16-bit depth instead of 32-bit
            codec='libmp3lame',  # Use 16-bit PCM codec
            ffmpeg_params=[
                "-ac", "1",  # Convert to mono (1 channel)
                "-b:a", "64k"    # Bitrate of 64kbps (you can adjust: 32k, 64k, 96k, 128k)
            ]
        )
        video.close()
        return audio_path

    # def transcribe_audio(audio_path):
    #     """Generate transcription from audio file"""
    #     recognizer = sr.Recognizer()

    #     with sr.AudioFile(audio_path) as source:
    #         audio = recognizer.record(source)

    #     try:
    #         transcription = recognizer.recognize_google(audio)

    #         # Save transcription to file
    #         with open(f"{output_dir}/transcription.txt", "w") as f:
    #             f.write(transcription)

    #         return transcription
    #     except sr.UnknownValueError:
    #         return "Speech could not be understood"
    #     except sr.RequestError:
    #         return "Could not request results from speech recognition service"

    # Execute analysis
    print("Extracting frames...")
    num_frames = extract_frames()
    print(f"Extracted {num_frames} frames")

    print("Extracting audio...")
    audio_path = extract_audio()
    print("Audio extracted successfully")

    print("Generating transcription...")
    # transcription = transcribe_audio(audio_path)
    print("Analysis complete!")

    return {
        "num_frames": num_frames,
        "audio_path": audio_path,
        # "transcription": transcription
    }

# Example usage


In [5]:
video_path = "MyVideo.mp4"
results = analyze_video(video_path)
print(f"Processed {results['num_frames']} frames")
print(f"Audio saved to: {results['audio_path']}")
# print(f"Transcription: {results['transcription'][:100]}...")

Extracting frames...
Total Extracted 6 frames
Extracted 1198 frames
Extracting audio...
MoviePy - Writing audio in output/audio.mp3


MoviePy - Done.
Audio extracted successfully
Generating transcription...
Analysis complete!
Processed 1198 frames
Audio saved to: output/audio.mp3


In [1]:
from openai import OpenAI
import os

audio_file= open("output/audio.wav", "rb")

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
transcription = client.audio.transcriptions.create( model="whisper-1", file=audio_file, response_format='verbose_json',timestamp_granularities=['word'])
print(transcription.text)


FileNotFoundError: [Errno 2] No such file or directory: 'output/audio.wav'